```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 问题

#h(2em) （原油采购与加工）某公司用两种原油（A 和 B）混合加工成两种汽油（甲和乙）。甲、乙两种汽油含原油A的最低比例分别为 50% 和 60%，每吨售价分别为 4800 元和 5600 元。该公司现有原油 A 和 B 的库存量分别为 500t 和 1000t，还可以从市场上买到不超过 1500t 的原油 A。原油 A 的市场价为：购买量不超过 500t 时的单价为 10000元/t；购买量超过 500t 但不超过 1000t 时，超过 500t 的部分 8000 元/t；购买量超过 1000t 时，超过 1000t 的部分 6000元/t。该公司应如何安排原油的采购和加工？请分别建立连续规划和整数规划模型来求解这个问题。

= 思路

== 连续规划

#h(2em) 将原油 A 采购量的 x 分为三个量，用 $x_1$，$x_2$，$x_3$ 分别表示 10 千元每吨，8 千元每吨，6 千元每吨采购的原油 A 的吨数，则支出为 $c(x) = 10 * x_1 + 8 * x_2 + 6 * x_3$ 千元，有 $x = x_1 + x_2 + x_3$。设甲由 $x_11$ 的 A 和 $x_12$ 的 B 混合而成，乙由 $x_21$ 的 A 和 $x_22$ 的 B 混合而成。

$
  "target" = 4.8 (x_11 + x_12) + 5.6 (x_21 + x_22) - c(x)
$

同时引入等式限制条件，代码实现如下：

```


In [ ]:
import numpy as np

In [ ]:
# 连续规划
from scipy.optimize import minimize as mini, NonlinearConstraint as NLC


def target(a):
  temp = 4.8 * sum(a[0:2]) + 5.6 * sum(a[2:4]) - (10 * a[4] + 8 * a[5] + 6 * a[6])
  return -temp


nlc = [
  NLC(lambda a: (a[4] - 500) * a[5], 0, 0),
  NLC(lambda a: (a[5] - 500) * a[6], 0, 0),
  NLC(lambda a: a, np.zeros(7), np.ones(7) * np.inf),
  NLC(lambda a: sum(a[4:7]) + 500 - a[0] - a[2], 0, 2000),
  NLC(lambda a: a[1] + a[3], 0, 1000),
  NLC(lambda a: a[0] - a[1], 0, np.inf),
  NLC(lambda a: 0.4 * a[2] - 0.6 * a[3], 0, np.inf),
]

res = mini(target, np.array([0, 0, 0, 0, 500, 500, 500]), constraints=nlc)

print(res)

```{=typst}

== 整数规划

引入 0-1 变量 z，其余思路同上。

代码如下所示：

```


In [ ]:
# 整数规划
import gurobipy as gp

i = gp.GRB.INTEGER
b = gp.GRB.BINARY
n = gp.GRB.MAXIMIZE

M = gp.Model
m = M('m')
a = m.addConstr
A = m.addConstrs
v = m.addVars
s = m.setObjective
S = m.Status
r = m.optimize
x = v(7, vtype=i, name='x')
z = v(3, vtype=b, name='z')

o = 4.8 * (x[0] + x[1]) + 5.6 * (x[2] + x[3]) - 10 * x[4] - 8 * x[5] - 6 * x[6]

s(o, n)

a(x[0] - x[1] >= 0, name='c1')
a(0.4 * x[2] - 0.6 * x[3] >= 0, name='c2')

a(x[1] + x[3] <= 1000, name='c3')
a(x[4] + x[5] + x[6] + 500 - x[0] - x[2] >= 0, name='c4')

A((x[i + 4] - 500 * z[i + 1] >= 0 for i in range(2)), name='c5')
A((x[i + 4] - 500 * z[i] <= 0 for i in range(3)), name='c6')
A((x[i] >= 0 for i in range(7)), name='c7')

r()

print('obj =', m.objVal)
print('x =', [x[i].x for i in range(7)])

购买 1000 吨原油 A，与库存的 500 吨原油 A 和 1000 吨原油 B 一起，共生产 2500 吨汽油乙，利润为 5000 （千元）。
